In [ ]:
import random

import numpy as np 
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch import nn 
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [ ]:
import random
import numpy as np
import os
SEED_VAL  = 1000
# Set the seed value all over the place to make this reproducible.
def seed_all(SEED):
  random.seed(SEED_VAL)
  np.random.seed(SEED_VAL)
  torch.manual_seed(SEED_VAL)
  torch.cuda.manual_seed_all(SEED_VAL)
  os.environ['PYTHONHASHSEED'] = str(SEED_VAL)
  torch.backends.cudnn.deterministic = True
seed_all(SEED_VAL)

In [ ]:
train_df = pd.read_csv("https://storage.googleapis.com/umojahack2022/train.csv")
test_df = pd.read_csv("https://storage.googleapis.com/umojahack2022/test.csv")

In [ ]:
test_df.tail()

Each row in the dataset represents a k-mer (16 amino acid sequence within the toxin) and it has a signal column coming from the high-density peptide microarray experiment. The dataframe has the following columns :
```
ID: Unique identifier for each row 
Toxin_UniprotID: Identifier for a specific toxin sequence in the Uniprot Database
Kmer_Position_start: The start position in the toxin global sequence of the Kmer_Position_end: The end  position in the toxin global sequence a given k-mer 
Antivenom: Name of the antivenom tested in the high-density peptide microarray experiment
Toxin_Kmer: String of 16 amino acids (16-mer, K=16) from a given toxin sequence
Signal: (target) The output of the experiment. A proxy for antivenom activity.
Genus: Genus of snake the toxin stems from, e.g. Naja (cobra)
Species: Species of snake the toxin originates from e.g. Naja nigricollis (Black-necked spitting cobra)
ProteinFam: Toxin protein family, e.g. three finger toxin (3FTx)
ProteinSubFam: Toxin sub-family, e.g. cytotoxin (a type of 3FTx)
ProteinSubSubFam: Toxin sub-sub-family, e.g. cytotoxin IA (a type of cytotoxin)
```

We can use any of these colums to train our ML model.

For our model, we will use the `Antivenom` and the `Toxin_Kmer` and the `Kmer_Position_start` columns.

We will first create our maps, which converts amino acids of the Toxin Kmer sequence and the Antivenom classes to numerical values

In [ ]:
def get_seq_column_map(train, test, col):
    sequences = []
    for seq in train[col]:
        sequences.extend(list(seq))
    for seq in test[col]:
        sequences.extend(list(seq))
    unique = np.unique(sequences)
    return {k: v for k, v in zip(unique, range(len(unique)))}

def get_column_map(train, test, col):
    sequences = []
    unique_values = pd.concat([train[col], test[col]]).unique().tolist()
    return {k: v for k, v in zip(unique_values, range(len(unique_values)))}

In [ ]:
amino_acid_map = get_seq_column_map(train_df, test_df, "Toxin_Kmer")
print("unique amino acid map",len(amino_acid_map))

antivenom_map = get_column_map(train_df, test_df, "Antivenom")
print("unique Antivenom map", len(antivenom_map))

We will split the data into a training and a validation set

We look at the GPU provided by Colab

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device : {device}")
torch.cuda.get_device_name()

We convert our data into a torch `Dataset`.
All datasets that represent a map from keys to data samples should subclass
`Dataset`. All subclasses should overwrite `__getitem__`, supporting fetching a data sample for a given key:

In [ ]:
class AntivenomChallengeDataSet(Dataset):
    def __init__(
        self,
        amino_acid_map,
        antivenom_map,
        data,
        is_train,
        label_name=None,
      ):
        self.amino_acid_map = amino_acid_map
        self.antivenom_map = antivenom_map
        self.data = data
        self.is_train = is_train
        self.label_name = label_name

    def __len__(self):
        return len(self.data) 

    def __getitem__(self,idx):
        row = self.data.iloc[idx]
        kmer_seq = torch.as_tensor([self.amino_acid_map[e] for e in list(row["Toxin_Kmer"])])
        antivenom = torch.as_tensor(self.antivenom_map[row["Antivenom"]])
        position_start = torch.as_tensor(row["Kmer_Position_start"])
        position_end = torch.as_tensor(row["Kmer_Position_end"])
        
        inputs = {
            "K_mer": kmer_seq,
            "antivenom": antivenom,
            "position_start": position_start,
            "position_end": position_end,
        }

        if self.is_train: 
            return inputs, torch.as_tensor([row[self.label_name]])
        return inputs

In [ ]:

test_dataset = AntivenomChallengeDataSet(
    amino_acid_map=amino_acid_map,
    antivenom_map=antivenom_map,
    data=test_df,
    is_train=False,
)

In [ ]:
batch_size = 128
num_workers = 0
shuffle = True
drop_last = False

Now we create our PyTorch data loaders. These combine a dataset and a sampler, and provide an iterable over the given dataset.

In [ ]:
test_data_loader= DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False,
)

## Define the model
For this example we will build an LSTM architeture. It is your task to come up with more performant architectures to improve the scores.

In [ ]:
class SimpleSeqModel(nn.Module):
    def __init__(
        self,
        K_mer_emb_size,
        K_mer_nunique,
        antivenom_emb_size,
        antivenom_unique,
        max_Position_start,
        Position_start_emb_size,
    ): 
        super().__init__()
        self.K_mer_emb_size = K_mer_emb_size        
        self.K_mer_nunique = K_mer_nunique                
        self.antivenom_emb_size = antivenom_emb_size  
        self.antivenom_unique = antivenom_unique    
        
        self.Kmer_emb_layer = nn.Embedding(
            num_embeddings=self.K_mer_nunique,
            embedding_dim=self.K_mer_emb_size,
        )
        self.Antivenom_emb = nn.Embedding(
            num_embeddings=self.antivenom_unique,
            embedding_dim=self.antivenom_emb_size,
        )
    
        self.Position_start_emb = nn.Embedding(
            num_embeddings=max_Position_start,
            embedding_dim=Position_start_emb_size,
        )
        self.Features = nn.Linear(
            in_features=self.antivenom_emb_size + Position_start_emb_size,
            out_features=128,
        )
        
        self.Lstm_layer_1 = nn.LSTM(
            input_size=self.K_mer_emb_size,
            hidden_size=256,
            num_layers=1,
            bidirectional=True,
            batch_first=True,
        )
        self.Lstm_layer_2 = nn.GRU(
            input_size=512,
            hidden_size=256,
            num_layers=1,
            bidirectional=False,
            batch_first=True,
        )
        
        self.Linear_1 = nn.Linear(
            in_features=self.Lstm_layer_2.hidden_size + self.Features.out_features,
            out_features=512,
        )
        self.relu_1 = nn.ReLU()
        self.Linear_2 = nn.Linear(
            in_features=self.Linear_1.out_features, out_features=256,
        )
        self.relu_2 = nn.ReLU()
        self.Output = nn.Linear(
            in_features=self.Linear_2.out_features, out_features=1,
        )
        
    def forward(self, inputs):
        kmer_emb = self.Kmer_emb_layer(inputs["K_mer"])
        antivenom_emb = self.Antivenom_emb(inputs["antivenom"])
        position_start_emb = self.Position_start_emb(inputs["position_start"])

        emb_features = torch.cat((antivenom_emb, position_start_emb), axis=1)
        features = self.Features(emb_features)
        
        lstm_1_seq, (lstm_1_h, lstm1_c) = self.Lstm_layer_1(kmer_emb)
        lstm_2_seq, lstm_2_h = self.Lstm_layer_2(lstm_1_seq)

        lstm_h = torch.squeeze(lstm_2_h)
        emb = torch.cat((lstm_h, features), axis=1)
        linear_1 = self.relu_1(self.Linear_1(emb))
        linear_2 = self.relu_2(self.Linear_2(linear_1))
        output = self.Output(linear_2)
        return output
        
        

Now that the model architecture is defined we are goint to instantiate our model. For this we need to calculate `max_Position_start` in order to calculate the size of the embedding layer we will use to encode the start position. The maximum position that the train and test dataset can have is:


In [ ]:
max_Position_start = pd.concat([train_df[["Kmer_Position_start"]], test_df[["Kmer_Position_start"]]]).Kmer_Position_start.max()+1

print(f"Max Position_start : {max_Position_start}")


### Training the model
We define a simple training loop


In [ ]:
def train_func(
    train_data_loader,
    val_data_loader,
    model,
    loss_fn,
    optimizer,
    num_epochs,
    device,
    early_stopping=5,
): 
    total_batches = len(train_data_loader)
    total_batches_val = len(val_data_loader)
    train_loss = []
    all_rmse=[]
    n_iter = 0
    for epoch in range(num_epochs): 
        tqdm_bar = tqdm(train_data_loader, desc=f"epoch {epoch}", position=0) 
        old_val_loss = np.inf
        wating = 0
        model.train()
        for batch_number, (X, y) in enumerate(tqdm_bar):
            y = y.type(torch.FloatTensor).to(device)
            X = {k: X[k].to(device) for k in X}
            
            optimizer.zero_grad()
            pred = model(X)
            loss = loss_fn(pred, y)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            loss = loss.item()
            train_loss.append(loss)

            n_iter += 1

            if batch_number % 25 == 0: 
                tqdm_bar.set_postfix(
                    {
                        "train": f"{batch_number}/{total_batches} loss: {loss:.3} epoch loss: {np.mean(train_loss):.3}",
                    },
                )

        val_tqdm_bar = tqdm(
            val_data_loader, desc=f"epoch {epoch}", position=0, leave=True,
        ) 
        val_loss = []
        val_rmse=[]
        model.eval()
        with torch.no_grad(): 
            for batch_number, (X, y) in enumerate(val_tqdm_bar):
                y = y.type(torch.FloatTensor).to(device)
                X = {k: X[k].to(device) for k in X}
                
                pred = model(X)
                val_loss.append(loss_fn(pred, y).item())
                val_rmse.append(mean_squared_error(pred.cpu(),y.cpu(),squared=False))


                if batch_number % 25 == 0: 
                    val_tqdm_bar.set_postfix(
                        {
                            "valid": f"{batch_number}/{total_batches_val} val loss: {np.mean(val_loss):.3} val rmse: {np.mean(val_rmse):.3}"
                        },
                    )
        
        new_val_loss = np.mean(val_loss)

        if new_val_loss > old_val_loss:
            wating += wating
        else:
            old_val_loss = new_val_loss
            torch.save(model.state_dict(),f"best_model_{fold}") 

        if wating > early_stopping:
            break
    
        all_rmse.append(np.mean(val_rmse))
    return np.mean(all_rmse)

In [ ]:
LR=1e-3
num_epochs = 10
early_stopping = 5

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True,random_state=152)


In [ ]:
all_rmse_scores = []
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_df)):
  print(f"#########################  Fold {fold+1}/{kfold.n_splits}  #########################")
  train_split_df , val_split_df = train_df.iloc[train_ids,:],train_df.iloc[test_ids,:]

  train_dataset = AntivenomChallengeDataSet(
    amino_acid_map=amino_acid_map,
    antivenom_map=antivenom_map,
    data=train_split_df,
    is_train=True,
    label_name="Signal")

  val_dataset = AntivenomChallengeDataSet(
      amino_acid_map=amino_acid_map,
      antivenom_map=antivenom_map,
      data=val_split_df,
      is_train=True,
      label_name="Signal")
  
  train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=shuffle,
    num_workers=num_workers,
    drop_last=drop_last,)

  val_data_loader = DataLoader(
      dataset=val_dataset,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      drop_last=False,)
  
  model = SimpleSeqModel(
    K_mer_emb_size=512,
    K_mer_nunique=len(amino_acid_map),
    antivenom_emb_size=64,
    antivenom_unique=len(antivenom_map),
    max_Position_start=max_Position_start,
    Position_start_emb_size=64,)

  loss_fn = nn.MSELoss()

  model = model.to(device)

  optimizer = torch.optim.Adam(model.parameters(), lr=LR)
  rmse=train_func(
    train_data_loader=train_data_loader,
    val_data_loader=val_data_loader,
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device,
    early_stopping=early_stopping)
  print('RMSE: '+str(rmse))
  all_rmse_scores.append(rmse)


In [ ]:
np.mean(all_rmse_scores)

In [ ]:
0.4365119

### Sample baseline Submission
Finally we will prepare a baseline submission to Zindi 


In [ ]:
def predict_test(data_loader, path): 
  all_folds = []
  for i in range(kfold.n_splits):
      model = SimpleSeqModel(
      K_mer_emb_size=512,
      K_mer_nunique=len(amino_acid_map),
      antivenom_emb_size=64,
      antivenom_unique=len(antivenom_map),
      max_Position_start=max_Position_start,
      Position_start_emb_size=64,)
      model.load_state_dict(torch.load(f'best_model_{i}'))
      model.to(device)
      model.eval()
      tqdm_bar = tqdm(data_loader, desc="Inference", position=0, leave=True) 
      total_batches = len(tqdm_bar)

      preds = []
      with torch.no_grad():
          for batch_number, X in enumerate(tqdm_bar):
              X= {k: X[k].to(device) for k in X}
              pred = model(X)
              preds.append(pred.cpu().numpy())

          preds = np.concatenate(preds)
      all_folds.append(preds)
  return all_folds

In [ ]:
test_pred = predict_test(test_data_loader,"model.pth")

In [ ]:
out_sum = test_pred[0]
for i in range(1,len(test_pred)):
    out_sum = test_pred[i] + out_sum

In [ ]:
((out_sum)/len(test_pred)).reshape((-1))

In [ ]:
sample_submission=test_df[["ID"]]
sample_submission["Signal"] = ((out_sum)/len(test_pred)).reshape((-1))
sample_submission['Signal']=sample_submission['Signal'].clip(lower=-1)
sample_submission.to_csv("./sample_submission19.csv",index=False)

That is it! Now we can upload the sample_submission.csv to Zindi! As a final thing lets look at it. 